In [27]:
import requests
import pandas as pd
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import Ridge, Lasso


def get_bitcoin_data():
    url = "https://api.coingecko.com/api/v3/coins/bitcoin/market_chart"
    params = {
        'vs_currency': 'usd',
        'days': '365',
        'interval': 'daily'
    }
    response = requests.get(url, params=params)
    data = response.json()
    return data

def prepare_data():
    data = get_bitcoin_data()
    prices = data['prices']


    df = pd.DataFrame(prices, columns=['timestamp', 'price'])


    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)

    return df

df = prepare_data()
print(df.head())


                   price
timestamp               
2023-09-16  26634.631025
2023-09-17  26557.768692
2023-09-18  26520.988255
2023-09-19  26741.461111
2023-09-20  27219.296875


In [8]:
#Pré processamento dos dados. Pegando do dia anterior, dos ultimos 7 dias e a média dos últimos 7 dias
def create_features(df):
    # Criar lags de 1 e 7 dias
    df['lag_1'] = df['price'].shift(1)
    df['lag_7'] = df['price'].shift(7)


    df['ma_7'] = df['price'].rolling(window=7).mean()

    df = df.dropna()

    return df

df_features = create_features(df)
print(df_features.head())


                   price         lag_1         lag_7          ma_7
timestamp                                                         
2023-09-23  26572.038112  26561.133454  26634.631025  26755.504706
2023-09-24  26573.923480  26572.038112  26557.768692  26757.812533
2023-09-25  26249.562898  26573.923480  26520.988255  26719.037482
2023-09-26  26298.634678  26249.562898  26741.461111  26655.776563
2023-09-27  26204.757591  26298.634678  27219.296875  26510.842380


In [9]:
X = df_features[['lag_1', 'lag_7', 'ma_7']]
y = df_features['price']


In [23]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


Par RIDGE LASSO

In [46]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

alphas = [0.01, 0.1, 1.0, 10.0]
solvers = ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg']  
max_iters = [100, 500, 1000]  

ridge_metrics = {}
lasso_metrics = {}

for alpha in alphas:
    for solver in solvers:
        ridge_model = Ridge(alpha=alpha, solver=solver)
        ridge_model.fit(X_train, y_train)
        y_pred = ridge_model.predict(X_test)
        ridge_metrics[(alpha, solver)] = {
            'MSE': mean_squared_error(y_test, y_pred),
            'R²': r2_score(y_test, y_pred)
        }

for alpha in alphas:
    for max_iter in max_iters:
        lasso_model = Lasso(alpha=alpha, max_iter=max_iter)
        lasso_model.fit(X_train, y_train)
        y_pred = lasso_model.predict(X_test)
        lasso_metrics[(alpha, max_iter)] = {
            'MSE': mean_squared_error(y_test, y_pred),
            'R²': r2_score(y_test, y_pred)
        }

ridge_metrics_df = pd.DataFrame(ridge_metrics).T
lasso_metrics_df = pd.DataFrame(lasso_metrics).T

print("Métricas da Regressão Ridge:")
print(ridge_metrics_df)

print("\nMétricas da Regressão Lasso:")
print(lasso_metrics_df)

best_params_ridge = ridge_metrics_df['MSE'].idxmin()
best_params_lasso = lasso_metrics_df['MSE'].idxmin()

print("\nMelhores Parâmetros da Regressão Ridge:")
print(best_params_ridge)

print("\nMelhores Parâmetros da Regressão Lasso:")
print(best_params_lasso)


Métricas da Regressão Ridge:
                          MSE        R²
0.01  auto       2.446619e+06  0.986019
      svd        2.446619e+06  0.986019
      cholesky   2.446619e+06  0.986019
      lsqr       2.446619e+06  0.986019
      sparse_cg  2.446619e+06  0.986019
0.10  auto       2.446619e+06  0.986019
      svd        2.446619e+06  0.986019
      cholesky   2.446619e+06  0.986019
      lsqr       2.446619e+06  0.986019
      sparse_cg  2.446619e+06  0.986019
1.00  auto       2.446619e+06  0.986019
      svd        2.446619e+06  0.986019
      cholesky   2.446619e+06  0.986019
      lsqr       2.446619e+06  0.986019
      sparse_cg  2.446619e+06  0.986019
10.00 auto       2.446619e+06  0.986019
      svd        2.446619e+06  0.986019
      cholesky   2.446619e+06  0.986019
      lsqr       2.446619e+06  0.986019
      sparse_cg  2.446619e+06  0.986019

Métricas da Regressão Lasso:
                     MSE        R²
0.01  100   2.369525e+06  0.986459
      500   2.431496e+06  0.986

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.646e+08, tolerance: 4.801e+06
  model = cd_fast.enet_coordinate_descent(
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.547e+08, tolerance: 4.801e+06
  model = cd_fast.enet_coordinate_descent(
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-p

PAR GradientBoosting

In [47]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

n_estimators = [50, 100, 200]
max_depth = [3, 5, 7]
learning_rates = [0.01, 0.1, 0.2]
min_samples_split = [2, 5, 10]

gb_metrics = {}

for n in n_estimators:
    for depth in max_depth:
        for lr in learning_rates:
            for min_split in min_samples_split:
                gb_model = GradientBoostingRegressor(
                    n_estimators=n,
                    max_depth=depth,
                    learning_rate=lr,
                    min_samples_split=min_split
                )
                gb_model.fit(X_train, y_train)
                y_pred = gb_model.predict(X_test)
                gb_metrics[(n, depth, lr, min_split)] = {
                    'MSE': mean_squared_error(y_test, y_pred),
                    'R²': r2_score(y_test, y_pred)
                }

gb_metrics_df = pd.DataFrame(gb_metrics).T
print("Métricas do Gradient Boosting:")
print(gb_metrics_df)

best_params_gb = gb_metrics_df['MSE'].idxmin()
print("\nMelhores Parâmetros:")
print(best_params_gb)


Métricas do Gradient Boosting:
                        MSE        R²
50  3 0.01 2   6.724901e+07  0.615709
           5   6.724408e+07  0.615737
           10  6.722158e+07  0.615865
      0.10 2   3.325504e+06  0.980997
           5   3.333973e+06  0.980948
...                     ...       ...
200 7 0.10 5   3.659864e+06  0.979086
           10  3.623319e+06  0.979295
      0.20 2   3.833777e+06  0.978092
           5   3.912725e+06  0.977641
           10  3.444201e+06  0.980318

[81 rows x 2 columns]

Melhores Parâmetros:
(np.int64(100), np.int64(3), np.float64(0.1), np.int64(10))


xgboost

In [48]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
import pandas as pd

n_estimators = [50, 100, 200]
learning_rates = [0.01, 0.1, 0.2]
max_depth = [3, 5, 7]
subsample = [0.8, 0.9, 1.0]

xgb_metrics = {}

for n in n_estimators:
    for lr in learning_rates:
        for depth in max_depth:
            for subs in subsample:
                xgb_model = xgb.XGBRegressor(
                    objective='reg:squarederror',
                    n_estimators=n,
                    learning_rate=lr,
                    max_depth=depth,
                    subsample=subs
                )
                xgb_model.fit(X_train, y_train)
                y_pred = xgb_model.predict(X_test)
                xgb_metrics[(n, lr, depth, subs)] = {
                    'MSE': mean_squared_error(y_test, y_pred),
                    'R²': r2_score(y_test, y_pred)
                }

xgb_metrics_df = pd.DataFrame(xgb_metrics).T
print("Métricas do XGBoost:")
print(xgb_metrics_df)

best_params_xgb = xgb_metrics_df['MSE'].idxmin()
print("\nMelhores Parâmetros:")
print(best_params_xgb)


Métricas do XGBoost:
                         MSE        R²
50  0.01 3 0.8  7.058787e+07  0.596629
           0.9  7.021133e+07  0.598781
           1.0  6.990041e+07  0.600557
         5 0.8  7.029831e+07  0.598284
           0.9  6.983088e+07  0.600955
...                      ...       ...
200 0.20 5 0.9  3.702214e+06  0.978844
           1.0  3.791393e+06  0.978334
         7 0.8  3.961905e+06  0.977360
           0.9  3.547905e+06  0.979726
           1.0  3.651771e+06  0.979132

[81 rows x 2 columns]

Melhores Parâmetros:
(np.int64(50), np.float64(0.1), np.int64(3), np.float64(0.9))


MLPRegressor

In [ ]:
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
import pandas as pd

hidden_layer_sizes = [(50,), (100,), (100, 50), (100, 100), (50, 50, 50)]
activation = ['relu', 'tanh', 'logistic']
alpha = [0.0001, 0.001, 0.01]
solver = ['lbfgs', 'sgd', 'adam']

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

nn_metrics = {}

for layers in hidden_layer_sizes:
    for act in activation:
        for alpha_val in alpha:
            for solver_method in solver:
                try:
                    nn_model = MLPRegressor(
                        hidden_layer_sizes=layers,
                        activation=act,
                        alpha=alpha_val,
                        solver=solver_method,
                        max_iter=1000,
                        verbose=True,
                        learning_rate_init=0.01
                    )
                    nn_model.fit(X_train_scaled, y_train)
                    y_pred = nn_model.predict(X_test_scaled)
                    nn_metrics[(layers, act, alpha_val, solver_method)] = {
                        'MSE': mean_squared_error(y_test, y_pred),
                        'R²': r2_score(y_test, y_pred)
                    }
                except Exception as e:
                    print(f"Error with parameters {layers}, {act}, {alpha_val}, {solver_method}: {e}")

nn_metrics_df = pd.DataFrame(nn_metrics).T
print("Métricas do MLP Regressor:")
print(nn_metrics_df)

best_params_nn = nn_metrics_df['MSE'].idxmin()
print("\nMelhores Parâmetros:")
print(best_params_nn)


In [45]:
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import LinearRegression

#regressão linear
model = LinearRegression()
model.fit(X_train, y_train)

best_alpha_ridge = ridge_metrics_df['MSE'].idxmin()
best_alpha_lasso = lasso_metrics_df['MSE'].idxmin()

# Regressão Ridge
ridge_model = Ridge(alpha=best_alpha_ridge)
ridge_model.fit(X_train, y_train)

# Regressão Lasso
lasso_model = Lasso(alpha=best_alpha_lasso)
lasso_model.fit(X_train, y_train)

best_params_gb = gb_metrics_df['MSE'].idxmin()

best_n_estimators, best_max_depth = best_params_gb

# Gradient Boosting
gb_model = GradientBoostingRegressor(n_estimators=best_n_estimators, max_depth=best_max_depth)
gb_model.fit(X_train, y_train)

best_params_xgb = xgb_metrics_df['MSE'].idxmin()

best_n_estimators, best_learning_rate = best_params_xgb

# XGBoost
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=best_n_estimators, learning_rate=best_learning_rate)
xgb_model.fit(X_train, y_train)

best_params_nn = nn_metrics_df['MSE'].idxmin()

best_hidden_layer_sizes, best_activation = best_params_nn

# MLP Regressor
nn_model = MLPRegressor(hidden_layer_sizes=best_hidden_layer_sizes, activation=best_activation, max_iter=500)
nn_model.fit(X_train, y_train)

C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\linear_model\_coordinate_descent.py:697: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.523e+08, tolerance: 4.801e+06
  model = cd_fast.enet_coordinate_descent(


MLPRegressor(hidden_layer_sizes=(100, 50), max_iter=500)

In [55]:
last_row = df_features.iloc[-1]

new_data = [[last_row['lag_1'], last_row['lag_7'], last_row['ma_7']]]

tomorrow_price = model.predict(new_data)
print(f'Previsão do preço do Bitcoin para amanhã: {tomorrow_price[0]:.2f}')
tomorrow_price = ridge_model.predict(new_data)
print(f'Previsão do preço do Bitcoin para amanhã: {tomorrow_price[0]:.2f}')
tomorrow_price = lasso_model.predict(new_data)
print(f'Previsão do preço do Bitcoin para amanhã: {tomorrow_price[0]:.2f}')
tomorrow_price = gb_model.predict(new_data)
print(f'Previsão do preço do Bitcoin para amanhã: {tomorrow_price[0]:.2f}')
tomorrow_price = xgb_model.predict(new_data)
print(f'Previsão do preço do Bitcoin para amanhã: {tomorrow_price[0]:.2f}')
tomorrow_price = nn_model.predict(new_data)
print(f'Previsão do preço do Bitcoin para amanhã: {tomorrow_price[0]:.2f}')


Previsão do preço do Bitcoin para amanhã: 60368.45
Previsão do preço do Bitcoin para amanhã: 60368.45
Previsão do preço do Bitcoin para amanhã: 60368.85
Previsão do preço do Bitcoin para amanhã: 60444.01
Previsão do preço do Bitcoin para amanhã: 60446.72
Previsão do preço do Bitcoin para amanhã: 963.30


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but Ridge was fitted with feature names
  warnings.warn(
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but Lasso was fitted with feature names
  warnings.warn(
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\base.py:493: UserWarning: X does not have val